In [1]:
import sys
sys.path.insert(0, '../../')

In [2]:
import graph
import importlib
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import random
import schedule as sch
import topo
import typing
import utils
import yaml
from tqdm.notebook import tqdm

matplotlib.rc('font', family='Times New Roman', size=16)

In [9]:
def unit_size_cb(r: int):
    return 10000 * math.pow(10, random.randint(0, 1))

def gen_graphs(graph_count, source_selector_dict):
    source_selector = graph.SourceSelector(source_selector_dict)
    gen_args_list = [
            {
            "total_rank": random.randint(3, 7),
            "max_node_per_rank": random.randint(1, 4),
            "max_predecessors": random.randint(1, 3),
            "mi_cb": lambda: 1,
            "memory_cb": lambda: int(2e8),
            "unit_size_cb": unit_size_cb,
            "unit_rate_cb": lambda: random.randint(10, 20),
            "source_hosts": source_selector,
            "sink_hosts": ["cloud1"],
        }
        for _ in range(graph_count)
    ]
    return [
        graph.GraphGenerator("g" + str(idx), **gen_args).gen_dag_graph()
        for idx, gen_args in enumerate(gen_args_list)
    ]

In [4]:
def big_avg(array):
    return sum(array) / len(array)

In [5]:
def run(sc, labels, source_selector_dict):
    flow_bp_data = [[] for _ in labels]
    all_cloud_bp_data = [[] for _ in labels]
    flow_event_data = [[] for _ in labels]
    all_cloud_event_data = [[] for _ in labels]
    edge_random_event_data = [[] for _ in labels]
    test_round = 100
    pbar = tqdm(total=len(labels)*test_round)
    for idx, graph_count in enumerate(labels):
        for _ in range(test_round):
            graph_list = gen_graphs(graph_count, source_selector_dict)
            sc.topo.clear_occupied()
            flow_scheduler = sch.FlowScheduler(sc)
            flow_calculator = sch.LatencyCalculator(sc.topo)
            flow_result_list = flow_scheduler.schedule_multiple(graph_list)
            for g, result in zip(graph_list, flow_result_list):
                assert result is not None
                flow_calculator.add_scheduled_graph(g, result)
            flow_latency, flow_bp = flow_calculator.compute_latency()
            for g in graph_list:
                flow_event_data[idx].append(flow_latency[g.uuid] / g.number_of_vertices())
                flow_bp_data[idx].append(flow_bp[g.uuid])

            sc.topo.clear_occupied()
            all_cloud_scheduler = sch.AllCloudScheduler(sc)
            all_cloud_calculator = sch.LatencyCalculator(sc.topo)
            all_cloud_result_list = all_cloud_scheduler.schedule_multiple(graph_list)
            for g, result in zip(graph_list, all_cloud_result_list):
                assert result is not None
                all_cloud_calculator.add_scheduled_graph(g, result)
            all_cloud_latency, all_cloud_bp = all_cloud_calculator.compute_latency()
            for g in graph_list:
                all_cloud_event_data[idx].append(all_cloud_latency[g.uuid] / g.number_of_vertices())
                all_cloud_bp_data[idx].append(all_cloud_bp[g.uuid])

            pbar.update()
    flow_event_data = [big_avg(i) for i in flow_event_data]
    flow_bp_data = [big_avg(i) for i in flow_bp_data]
    all_cloud_event_data = [big_avg(i) for i in all_cloud_event_data]
    all_cloud_bp_data = [big_avg(i) for i in all_cloud_bp_data]
    return flow_event_data, flow_bp_data, all_cloud_event_data, all_cloud_bp_data

In [11]:
# sc = topo.Scenario.from_dict(yaml.load(open("../../samples/1e3h.yaml", "r").read(), Loader=yaml.Loader))
sc = topo.Scenario.from_dict(yaml.load(open("../../samples/1e12h.yaml", "r").read(), Loader=yaml.Loader))
# sc = topo.Scenario.from_dict(yaml.load(open("../../samples/1e40h.yaml", "r").read(), Loader=yaml.Loader))
graph_count = 120
# source_dict = {'rasp1': 8, 'rasp2': 8, 'rasp3': 8}
source_dict = {'rasp'+str(i): 8 for i in range(1, 7)}
source_dict.update({'vm'+str(i): 16 for i in range(1, 7)})
# source_dict = {'e0rasp'+str(i): 8 for i in range(1, 9)}
# source_dict.update({'e1rasp'+str(i): 8 for i in range(1, 9)})
# source_dict.update({'e0vm'+str(i): 16 for i in range(1, 9)})
# source_dict.update({'e1vm'+str(i): 16 for i in range(1, 9)})
# source_dict.update({'e0desktop'+str(i): 32 for i in range(1, 5)})
# source_dict.update({'e1desktop'+str(i): 32 for i in range(1, 5)})
graph_list = gen_graphs(graph_count, source_dict)
sc.topo.clear_occupied()
flow_scheduler = sch.FlowScheduler(sc)
flow_calculator = sch.LatencyCalculator(sc.topo)
flow_result_list = flow_scheduler.schedule_multiple(graph_list)
for g, result in zip(graph_list, flow_result_list):
    assert result is not None
    flow_calculator.add_scheduled_graph(g, result)
for u, v, data in sc.topo.g.edges(data=True):
    print('{} -> {}: {}'.format(u, v, data['occupied'] / 1e6))

cloud0_router -> cloud_switch: 325.66
cloud0_router -> edge0_router: 325.66
cloud_switch -> cloud1: 325.66
edge0_router -> rasp_switch: 114.27
edge0_router -> vm_switch: 211.39
rasp_switch -> rasp1: 18.21
rasp_switch -> rasp2: 22.79
rasp_switch -> rasp3: 22.89
rasp_switch -> rasp4: 15.15
rasp_switch -> rasp5: 9.38
rasp_switch -> rasp6: 35.65
vm_switch -> vm1: 33.61
vm_switch -> vm2: 33.02
vm_switch -> vm3: 32.55
vm_switch -> vm4: 55.69
vm_switch -> vm5: 41.6
vm_switch -> vm6: 43.52


## rerun & debug

In [6]:
importlib.reload(sch)
importlib.reload(sch.flow_provisioner)
sc = topo.Scenario.from_dict(yaml.load(open("../../samples/1e40h.yaml", "r").read(), Loader=yaml.Loader))
graph_list = graph.ExecutionGraph.load_all(open('./debug-graphs.yaml', 'r'))
sc.topo.clear_occupied()
flow_scheduler = sch.FlowScheduler(sc)
flow_calculator = sch.LatencyCalculator(sc.topo)
flow_result_list = flow_scheduler.schedule_multiple(graph_list)
for g, result in zip(graph_list, flow_result_list):
    assert result is not None
    flow_calculator.add_scheduled_graph(g, result)
for u, v, data in sc.topo.g.edges(data=True):
    print('{} -> {}: {}'.format(u, v, data['occupied'] / 1e6))

rebalance round 0
rebalance round 1
rebalance round 2
rebalance round 3
rebalance round 0
rebalance round 1
rebalance round 2
rebalance round 3
rebalance round 0
rebalance round 1
rebalance round 2
rebalance round 3
edge0 run provisioning
rebalance round 0
rebalance round 1
rebalance round 2
rebalance round 3
rebalance round 4
rebalance round 5
cloud0 run provisioning
rebalance round 0
rebalance round 1
rebalance round 2
rebalance round 3
edge1 run provisioning
rebalance round 0
rebalance round 1
rebalance round 2
rebalance round 3
rebalance round 4
rebalance round 5
cloud0 run provisioning
rebalance round 0
rebalance round 1
rebalance round 2
rebalance round 3
cloud0_router -> cloud_switch: 1699.88
cloud0_router -> edge0_router: 876.97
cloud0_router -> edge1_router: 822.91
cloud_switch -> cloud1: 1699.88
edge0_router -> e0rasp_switch: 202.46
edge0_router -> e0vm_switch: 383.42
edge0_router -> e0desktop_switch: 323.69
edge0_router -> edge1_router: 0.0
e0rasp_switch -> e0rasp1: 30.86
e0

In [7]:
sc.topo.clear_occupied()
edge_scheduler = sch.EdgeRandomScheduler(sc)
edge_calculator = sch.LatencyCalculator(sc.topo)
edge_result_list = edge_scheduler.schedule_multiple(graph_list)
for g, result in zip(graph_list, edge_result_list):
    assert result is not None
    edge_calculator.add_scheduled_graph(g, result)
for u, v, data in sc.topo.g.edges(data=True):
    print('{} -> {}: {}'.format(u, v, data['occupied'] / 1e6))

cloud0_router -> cloud_switch: 1699.88
cloud0_router -> edge0_router: 876.97
cloud0_router -> edge1_router: 822.91
cloud_switch -> cloud1: 1699.88
edge0_router -> e0rasp_switch: 237.1
edge0_router -> e0vm_switch: 421.15
edge0_router -> e0desktop_switch: 369.02
edge0_router -> edge1_router: 0.0
e0rasp_switch -> e0rasp1: 35.56
e0rasp_switch -> e0rasp2: 38.66
e0rasp_switch -> e0rasp3: 32.59
e0rasp_switch -> e0rasp4: 39.29
e0rasp_switch -> e0rasp5: 28.3
e0rasp_switch -> e0rasp6: 25.21
e0rasp_switch -> e0rasp7: 20.49
e0rasp_switch -> e0rasp8: 34.4
e0vm_switch -> e0vm1: 43.08
e0vm_switch -> e0vm2: 77.23
e0vm_switch -> e0vm3: 48.83
e0vm_switch -> e0vm4: 61.55
e0vm_switch -> e0vm5: 57.94
e0vm_switch -> e0vm6: 50.03
e0vm_switch -> e0vm7: 58.74
e0vm_switch -> e0vm8: 58.55
e0desktop_switch -> e0desktop1: 113.39
e0desktop_switch -> e0desktop2: 95.77
e0desktop_switch -> e0desktop3: 91.68
e0desktop_switch -> e0desktop4: 94.38
edge1_router -> e1rasp_switch: 183.75
edge1_router -> e1vm_switch: 358.49


In [13]:
print(graph_list[407].get_sources()[0].domain_constraint)
provisioner = flow_scheduler.get_provisioner("edge1")

{'host': 'e1vm5'}


In [14]:
for p_node in provisioner.tree.name_lookup_map.values():
    for v in p_node.scheduled_vertices:
        if v.uuid == 'g407-v21':
            print('found')

# Privisioner Debug

In [22]:
import schedule.flow_provisioner as prov
importlib.reload(prov)
max_round = 1000
source_dict = {'e0rasp'+str(i): 8 for i in range(1, 9)}
# source_dict.update({'e1rasp'+str(i): 8 for i in range(1, 9)})
source_dict.update({'e0vm'+str(i): 16 for i in range(1, 9)})
# source_dict.update({'e1vm'+str(i): 16 for i in range(1, 9)})
source_dict.update({'e0desktop'+str(i): 32 for i in range(1, 5)})
# source_dict.update({'e1desktop'+str(i): 32 for i in range(1, 5)})
pbar = tqdm(total=max_round)
while max_round > 0:
    # print('---', max_round, '---')
    sc.topo.clear_occupied()
    source_selector = graph.SourceSelector(source_dict)
    gen_args = {
            "total_rank": random.randint(3, 7),
            "max_node_per_rank": random.randint(2, 5),
            "max_predecessors": random.randint(1, 4),
            "mi_cb": lambda: 1,
            "memory_cb": lambda: int(2e8),
            "unit_size_cb": unit_size_cb,
            "unit_rate_cb": lambda: random.randint(10, 20),
            "source_hosts": source_selector,
            "sink_hosts": ["cloud1"],
        }
    g = graph.GraphGenerator("g", **gen_args).gen_dag_graph()
    # print(g.number_of_vertices())
    provisioner = prov.TopologicalProvisioner(sc.get_edge_domains()[0])
    op_set = set([v.uuid for v in g.get_vertices()]) - set([v.uuid for v in g.get_sinks()])
    provisioner.schedule(g.sub_graph(op_set, "g"))
    max_round -= 1
    pbar.update()

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:08<00:00, 14.54it/s]


In [23]:
with open('./debug-graphs.yaml', 'w') as f:
    graph.ExecutionGraph.save_all(graph_list, f)